# Tâche: prédire les propriétaires d'assurance maladie qui seront intéressés par l'assurance automobile

Une compagnie d'assurance a fourni une assurance maladie à ses clients. Maintenant, ils veulent un modèle pour prédire si les assurés (clients) de l'année dernière seront également intéressés par l'assurance automobile fournie par l'entreprise.

Avant de créer un modèle, explorons l'ensemble de données et obtenons des informations à partir des données. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,roc_auc_score
%matplotlib inline

train=pd.read_csv('../input/health-insurance-cross-sell-prediction/train.csv')
#test=pd.read_csv('../input/health-insurance-cross-sell-prediction/test.csv')

print("First five rows of training dataset are:")
train.head()

On nous donne deux fichiers csv dans cet ensemble de données: train et test. Pour l'EDA, je n'utiliserai que les données d'entraînement.

Jetons un coup d'œil aux types de données de différentes colonnes.

In [ ]:
train.info()

In [ ]:
print(f"Le jeu d'entraînement a {train.shape[0]} observations et {train.shape[1]} variables.")
print(f"Des données sont manquantes dans le jeu de données: {train.isnull().sum().any()}")

# EDA

Les fonctionnalités Response, Driving_License et Previously_Insured sont déjà encodées dans les données. Pour l'EDA, je les convertis en type d'objet pour une compréhension facile dans les visualisations. 

In [ ]:
train_eda = train.copy()
cols=['Driving_License','Previously_Insured','Response']
for col in cols:
    train_eda[col] = train_eda[col].map({0:'No',1:'Yes'})

## Variable cible : Response

In [ ]:
sns.countplot(train_eda['Response'],palette='rocket')
plt.title("Target variable Distribution in data");

On voit ici que nos données sont très déséquilibrées.

## Genre

In [ ]:
sns.countplot(train_eda['Gender'],palette='summer')
plt.title("Gender Distribution in data");

* On a plus d'échantillons pour le genre mâle.
* Il semblerait donc que les hommes soient davantage concernés par des problèmes de santé que les femmes (cette information ne permet pas d'en être certains, il faudrait peut-être un dataset plus complet pour en être sûr).

## Age

In [ ]:
print("Age distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Age",shade = True)
facetgrid.set(xlim = (0,train_eda["Age"].max()))
facetgrid.add_legend();

* Les jeunes, de moins de 30 ans ne sont pas intéressés par une assurance pour leur véhicule. Les principales raisons pourraient être le manque d'expérience, de maturité et le fait qu'ils ne possèdent pas encore de véhicules à coût important.
* Des personnes entre 30 et 60 ans semblent être davantage intéressées.

## Permis de conduire

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Driving_License'])

> ***Vous devriez toujours avoir un permis de conduire en conduisant***

* Ici aussi les observations majoritaires concernent des personnes possédant un permis de conduire.


## Distribution par région

In [ ]:
train_eda['Region_Code'].value_counts().plot(kind='barh',cmap='Accent',figsize=(12,10));

* La signification de ces codes n'a pas été fournie.
* La plupart des données sont collectées auprès de personnes vivant dans la région avec le code 28​.

## Précédemment assuré 

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Previously_Insured'])

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Previously_Insured']).plot(kind='bar');

* Ceux qui ont déjà une assurance ne sont pas intéressés. C'est un résultat plutôt évident.


## Age du véhicule

In [ ]:
plt.rcParams['figure.figsize']=(6,8)
color = ['yellowgreen','gold',"lightskyblue"]
train_eda['Vehicle_Age'].value_counts().plot.pie(y="Vehicle_Age",colors=color,explode=(0.02,0,0.3),startangle=50,shadow=True,autopct="%0.1f%%")
plt.axis('on');

In [ ]:
sns.countplot(train_eda['Vehicle_Age'],hue=train_eda['Response'],palette='autumn');

* Plus de la moitié des données (52%) concernent des échantillons dont l'âge du véhicule est compris entre 1 et 2 ans.
* Nous ne pouvons pas dire à partir du deuxième graphique que les personnes dont l'âge du véhicule est compris entre 1 et 2 ans sont plus intéressées car l'autre catégorie «> 2 ans» a très peu d'observations.


## Dommages au véhicule 

In [ ]:
pd.crosstab(train_eda['Response'], train_eda['Vehicle_Damage']).plot(kind='bar');

*  Customers who got his/her vehicle damaged in the past is more likely to be interested in insurance. May be because he has first-hand experience of its pros and cons.
* Ah! I want a version of 'Prevention is better than cure' for this situation.

## Annual Premium

In [ ]:
print("Annual Premium distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Annual_Premium",shade = True)
facetgrid.set(xlim = (0,train_eda["Annual_Premium"].max()))
facetgrid.add_legend();

* Je ne pense pas que cela donne beaucoup d'informations supplémentaires.
* Des valeurs aberrantes peuvent être présentes ici.

## PolicySalesChannel

In [ ]:
print("Policy_Sales_Channel distribution according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Policy_Sales_Channel",shade = True)
facetgrid.set(xlim = (0,train_eda["Policy_Sales_Channel"].max()))
facetgrid.add_legend();

* Ce graphique semble intéressant. Mais pour extraire clairement des informations, nous avons besoin de la signification de ces codes.

## Vintage

Number of Days, Customer a été associé à l'entreprise

In [ ]:
print("Vintage feature according to Response")
facetgrid = sns.FacetGrid(train_eda,hue="Response",aspect = 4)
facetgrid.map(sns.kdeplot,"Vintage",shade = True)
facetgrid.set(xlim = (0,train_eda["Vintage"].max()))
facetgrid.add_legend();

* Notre variable cible n'est pas très affectée par cette feature. Elle peut être abandonnée.

In [ ]:
print("Correlation matrix-")
plt.rcParams['figure.figsize']=(8,6)
sns.heatmap(train.corr(),cmap='Spectral');

In [ ]:
train.corr()[:-1]['Response'].sort_values().round(2)

Je supprimerai les features les moins corrélées pour la modélisation.

# Création d'un modèle de classification

In [ ]:
#creating a checkpoint
df4model = train.copy()
#dropping Vintage column as suggested by EDA
df4model.drop(['id','Vintage'],axis=1,inplace=True)
#checking target variable
df4model.Response.value_counts()

Pour équilibrer ces données, nous allons suréchantillonner la classe minoritaire en utilisant le rééchantillonnage de la bibliothèque sklearn.

Pour éviter toute fuite de données, je vais d'abord me diviser en sous-ensembles de train et de test, puis effectuer un suréchantillonnage. 

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(df4model.drop(['Response'], axis = 1), 
                                                    df4model['Response'], test_size = 0.2)

In [ ]:
print(f"Target variable disribution in train set: \n{y_train.value_counts()}\n\nand in test set: \n{y_test.value_counts()}")

## Suréchantillonnage

In [ ]:
#combining train features and target
df = pd.concat([X_train,y_train],axis=1)

from sklearn.utils import resample,shuffle
df_majority = df[df['Response']==0]
df_minority = df[df['Response']==1]
df_minority_upsampled = resample(df_minority,replace=True,n_samples=y_train.value_counts()[0],random_state = 123)
balanced_df = pd.concat([df_minority_upsampled,df_majority])
balanced_df = shuffle(balanced_df)
balanced_df.Response.value_counts()

Maintenant, je vais convertir les colonnes catégorielles en colonnes numériques.

In [ ]:
from sklearn.preprocessing import OrdinalEncoder
encoder= OrdinalEncoder()
cat_cols=['Gender','Vehicle_Damage']
balanced_df[cat_cols] = encoder.fit_transform(balanced_df[cat_cols])
X_test[cat_cols] = encoder.transform(X_test[cat_cols])

dummy = pd.get_dummies(balanced_df['Vehicle_Age'],drop_first=True)
features = pd.concat([dummy,balanced_df],axis=1)
features.drop('Vehicle_Age',axis=1,inplace=True)

features.head()

In [ ]:
#to get uniform output
features = features.astype('float64')
X_train = features.drop('Response',axis=1)
y_train = features['Response']

#creating dummies in test set
dummy1 = pd.get_dummies(X_test['Vehicle_Age'],drop_first=True)
X_test = pd.concat([dummy1,X_test],axis=1)
X_test.drop('Vehicle_Age',axis=1,inplace=True)

# Régression Logistique

In [ ]:
logisticRegression = LogisticRegression(max_iter = 10000)
logisticRegression.fit(X_train, y_train)
predictions = logisticRegression.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,predictions).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,predictions).round(2)}")

# Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=100)
rfc.fit(X_train, y_train)
rfc_pred = rfc.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,rfc_pred).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,rfc_pred).round(2)}")

In [ ]:
rfc_preds = rfc.predict_proba(X_test)
print("AUC score after taking probabilities predictions and not classes predictions is")
roc_auc_score(y_test, rfc_preds[:,1], average = 'weighted')

In [ ]:
X_train.columns= ['less than 1 Year','greater than 2 Years', 'Gender', 'Age','Driving_License',
                  'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel']
X_test.columns= ['less than 1 Year','greater than 2 Years', 'Gender', 'Age','Driving_License',
                  'Region_Code', 'Previously_Insured', 'Vehicle_Damage', 'Annual_Premium','Policy_Sales_Channel']

from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
xgb_pred = xgb.predict(X_test)
print(f"Accuracy score is {100*accuracy_score(y_test,xgb_pred).round(2)}\nROC-AUC score is {100*roc_auc_score(y_test,xgb_pred).round(2)}")

In [ ]:
xgb_preds = xgb.predict_proba(X_test)
roc_auc_score(y_test, xgb_preds[:,1], average = 'weighted')